## Trainer Code

This notebook takes trainer video, which will be ideal for exercise and processing too. 
Trainer video will directly start the exercise, so initial position will be 0th frame
Trainer video contains side view of trainer and isn't tilted
Trainer will do one round of exercise only
The points and angles obtained will be stored in csv and used to compare with user frames for counting no of correct reps. 

## Import Libraries

In [1]:
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

pose_model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = pose_model.signatures['serving_default']
key_frames_angles = []
key_frames_points = []

#### Constants

In [2]:
threshold_angle = 45

#### Set exercise name

In [3]:
excersise_name = "PushUp"
if not os.path.exists(excersise_name):
    os.makedirs(excersise_name)
    print(f"Directory '{excersise_name}' created.")

Directory 'PushUp' created.


### Get angles and key points

In [4]:
### Returns key points by taking model output on a image
def get_keypoints(outputs):
    points = outputs['output_0'].numpy()[0, 0, :, :]
    nose = points[0]
    left_shoulder = points[5]
    right_shoulder = points[6]
    left_elbow = points[7]
    right_elbow = points[8]
    left_wrist = points[9]
    right_wrist = points[10]
    left_hip = points[11]
    right_hip = points[12]
    left_knee = points[13]
    right_knee = points[14]
    left_ankle = points[15]
    right_ankle = points[16]
    neck = (left_shoulder + right_shoulder) / 2

    body_parts = {
        'nose': nose,
        'left_shoulder': left_shoulder,
        'right_shoulder': right_shoulder,
        'left_elbow': left_elbow,
        'right_elbow': right_elbow,
        'left_wrist': left_wrist,
        'right_wrist': right_wrist,
        'left_hip': left_hip,
        'right_hip': right_hip,
        'left_knee': left_knee,
        'right_knee': right_knee,
        'left_ankle': left_ankle,
        'right_ankle': right_ankle,
        'neck': neck
    }
    return body_parts

In [5]:
def cosine_angle(v1, v2):
    numer = np.dot(v1, v2)
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)
    if denom == 0:
        if np.array_equal(v1, v2):
            return 0.0
        else:
            return 180.0
    degrees = np.degrees(np.arccos(numer / denom))
    return degrees

def calculate_angle(point1,point2,point3):
    v1 = point1[:2] - point2[:2]
    v2 = point3[:2] - point2[:2]
    return cosine_angle(v1, v2)

In [6]:
## Get angles from key points. 
def get_angles(keypoints):
    n_n_rs = calculate_angle(keypoints['nose'], keypoints['neck'], keypoints['right_elbow'])
    n_n_ls = calculate_angle(keypoints['nose'], keypoints['neck'], keypoints['left_elbow'])
    n_rs_re = calculate_angle(keypoints['neck'], keypoints['right_shoulder'], keypoints['right_elbow'])
    n_ls_le = calculate_angle(keypoints['neck'], keypoints['left_shoulder'], keypoints['left_elbow'])
    rs_re_rw = calculate_angle(keypoints['right_shoulder'], keypoints['right_elbow'], keypoints['right_wrist'])
    ls_le_lw = calculate_angle(keypoints['left_shoulder'], keypoints['left_elbow'], keypoints['left_wrist'])
    n_rh_rk = calculate_angle(keypoints['neck'], keypoints['right_hip'], keypoints['right_knee'])
    n_lh_lk = calculate_angle(keypoints['neck'], keypoints['left_hip'], keypoints['left_knee'])
    rh_rk_ra = calculate_angle(keypoints['right_hip'], keypoints['right_knee'], keypoints['right_ankle'])
    lh_lk_la = calculate_angle(keypoints['left_hip'], keypoints['left_knee'], keypoints['left_ankle'])

    angles = {
        "Nose-Neck-Right Shoulder": n_n_rs,
        "Nose-Neck-Left Shoulder": n_n_ls,
        "Neck-Right Shoulder-Right Elbow": n_rs_re,
        "Neck-Left Shoulder-Left Elbow": n_ls_le,
        "Right Shoulder-Right Elbow-Right Wrist": rs_re_rw,
        "Left Shoulder-Left Elbow-Left Wrist": ls_le_lw,
        "Neck-Right Hip-Right Knee": n_rh_rk,
        "Neck-Left Hip-Left Knee": n_lh_lk,
        "Right Hip-Right Knee-Right Ankle": rh_rk_ra,
        "Left Hip-Left Knee-Left Ankle": lh_lk_la
    }

    return angles

In [7]:
# Takes input as image path and return all angles and points
def get_data_from_model(image):

    image_tensor = tf.image.resize_with_pad(image, 192, 192)
    image_tensor = tf.cast(image_tensor, dtype=tf.int32)
    image_tensor = tf.expand_dims(image_tensor, axis=0)

    # Run the model.
    outputs = movenet(image_tensor)
    keypoints = get_keypoints(outputs)
    angles = get_angles(keypoints)
    return keypoints, angles

# Gets angles of current frame and compares wrt prev frame and saves if there is significant difference
def process_frame(frame, count, angles):
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = tf.convert_to_tensor(image,dtype=tf.int32)
    keypoints, angles1 = get_data_from_model(image)  # Pass the image to the model
    angle_diff = sum(abs(angles[key] - angles1[key]) for key in angles)
    if angle_diff > threshold_angle:
        filename = f"pose{count}.jpg"
        filename = os.path.join(excersise_name,filename)
        cv2.imwrite(filename, frame)
        angles = angles1
        key_frames_angles.append(angles)
        key_frames_points.append(keypoints)
    return angles

In [12]:
def main():
    video = cv2.VideoCapture("./GymBro/data/lunges.gif")
    count = 0
    ret, frame = video.read()
    
    if not ret:
        print("Failed to read video")
        return
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = tf.convert_to_tensor(image,dtype=tf.int32)
    
    _, angles = get_data_from_model(image)
    count += 1
    
    while True:
        ret, frame = video.read()
        if not ret:
            break
        angles = process_frame(frame, count, angles)
        
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [14]:
import csv
csv_file1 = "key_frame_angles.csv"
csv_file2 = "key_frame_points.csv"
csv_file1 = os.path.join(excersise_name,csv_file1)
csv_file2 = os.path.join(excersise_name,csv_file2)
# Extract the headers from the first dictionary (keys of the dictionary)
headers1 = key_frames_angles[0].keys()
headers2 = key_frames_points[0].keys()

# Write data to CSV
with open(csv_file1, "w", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers1)
    writer.writeheader()
    writer.writerows(key_frames_angles)
# Write data to CSV
with open(csv_file2, "w", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers2)
    writer.writeheader()
    writer.writerows(key_frames_points)